In [1]:
import pandas as pd
import logging
import re
import os
import time 
import pyaudio
import wave
from pydub import AudioSegment
from deep_translator import GoogleTranslator
from langdetect import detect
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from PyPDF2 import PdfFileReader
from docx import Document
from pptx import Presentation
from xlrd import open_workbook
from csv import reader
import json
import xml.etree.ElementTree as ET
import azure.cognitiveservices.speech as speechsdk

# Enable logging for debugging
logging.basicConfig(level=logging.DEBUG)


# Azure Custom Vision setup
ENDPOINT = "https://p2ic-prediction.cognitiveservices.azure.com/"
prediction_key = "64bfcfb8949d484fae93402fa55e1425"
project_id = "c450da0e-a657-4c10-a1a4-f8ed35e8d40a"
publish_name = "DiseaseClassification"

# Create CustomVisionPredictionClient
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, credentials)

# Azure credentials
AI_SERVICE_ENDPOINT = "https://p2languageservice.cognitiveservices.azure.com/"
AI_SERVICE_KEY = "741f912d0fb040d1afb1a3ca8b4f64d9"
TRANSLATION_ENDPOINT = "https://api.cognitive.microsofttranslator.com/"
TRANSLATION_KEY = "256e3786ca11469980ddae934f5944e5"
SPEECH_KEY = "872d1354c89e49158adb2b10b0c42493"
SPEECH_REGION = "eastus"

# Load all the datasets into a list of DataFrames
datasets = [
    pd.read_csv("D:/AI-Projects/FinalProject/P2/Lifestyle-Related Diseases.csv"),
    pd.read_csv("D:/AI-Projects/FinalProject/P2/disease_data.csv"),
    pd.read_csv("D:/AI-Projects/FinalProject/P2/Environmental Diseases.csv"),
    pd.read_csv("D:/AI-Projects/FinalProject/P2/Idiopathic.csv"),
    pd.read_csv("D:/AI-Projects/FinalProject/P2/Neoplastic Diseases.csv"),
    pd.read_csv("D:/AI-Projects/FinalProject/P2/non-infectious diseases_data.csv"),
    pd.read_csv("D:/AI-Projects/FinalProject/P2/Nutritional Diseases.csv"),
    pd.read_csv("D:/AI-Projects/FinalProject/P2/Psychiatric and Neurological Disorders.csv"),
    pd.read_csv("D:\AI-Projects\FinalProject\P2\Rare Diseases.csv")
]

# Combine all datasets into one DataFrame for easier searching
combined_df = pd.concat(datasets, ignore_index=True)

# Clean and normalize text
def clean_text(text):
    text = re.sub(r'\W+', ' ', text.lower())
    text = ' '.join(dict.fromkeys(text.split()))  # Remove duplicate words
    return text



def authenticate_azure_translation():
    credential = AzureKeyCredential(TRANSLATION_KEY)
    client = TextTranslationClient(endpoint=TRANSLATION_ENDPOINT, credential=credential)
    return client

def authenticate_azure_nlp():
    credential = AzureKeyCredential(AI_SERVICE_KEY)
    client = TextAnalyticsClient(endpoint=AI_SERVICE_ENDPOINT, credential=credential)
    return client

def detect_language(text):
    client = authenticate_azure_nlp()
    try:
        response = client.detect_language(documents=[text])
        primary_language = response[0].primary_language.iso6391_name
        return primary_language
    except Exception as e:
        print(f"Error detecting language: {e}")
        return None

def translate_to_english(text):
    client = authenticate_azure_translation()
    try:
        # Send the request to the Azure Translation service
        response = client.translate(
            body=[{
                "text": text
            }],
            to_language=["en"]
        )
        
        # Extract the translated text from the response
        if response and 'translations' in response[0]:
            translated_text = response[0]['translations'][0]['text']
            print(f"Translated Text: {translated_text}")
            return translated_text
        else:
            print("No translations found in the response.")
            return text
        
    except Exception as e:
        print(f"Error translating text: {e}")
        return text

# Translate text to English if not already in English
def translate_to_english(text):
    try:
        user_lang = detect(text)
        print(f"Detected Language: {lang_code_to_full.get(user_lang, user_lang)}")
        if user_lang != 'en':
            text = GoogleTranslator(source=user_lang, target='en').translate(text)
    except Exception as e:
        print(f"Error translating to English: {e}")
    return text




# Extract key phrases from user input text using Azure Text Analytics
def extract_key_phrases(text):
    client = authenticate_azure_nlp()
    response = client.extract_key_phrases(documents=[text])[0]
    return response.key_phrases

# Find diseases based on extracted key phrases (symptoms)
def find_disease_by_symptoms(key_phrases, combined_df):
    key_phrases_cleaned = [clean_text(phrase) for phrase in key_phrases]
    matched_diseases = combined_df[combined_df['Symptoms'].apply(lambda x: any(phrase in clean_text(x) for phrase in key_phrases_cleaned))]
    return matched_diseases

# Prioritize diseases by severity and frequency of key phrase matches
def prioritize_combined(matched_diseases, key_phrases):
    severity_priority = {'Mild': 1, 'Moderate': 2, 'Severe': 3}
    matched_diseases.loc[:, 'Severity Priority'] = matched_diseases['Severity Level'].map(severity_priority)
    
    def count_matches(symptoms):
        cleaned_symptoms = clean_text(symptoms)
        return sum(phrase in cleaned_symptoms for phrase in key_phrases)
    
    matched_diseases.loc[:, 'Match Count'] = matched_diseases['Symptoms'].apply(count_matches)
    prioritized_diseases = matched_diseases.sort_values(by=['Match Count', 'Severity Priority'], ascending=[False, False])
    return prioritized_diseases

# Print disease information in a formatted way
def print_disease_info(disease, language):
    disease_info = (
        f"\n--- Disease Information ---\n"
        f"Disease Name: {disease['Disease Name']}\n"
        f"Severity Level: {disease['Severity Level']}\n"
        f"Symptoms: {disease['Symptoms']}\n"
        f"Recommended Medications: {disease['Recommended Medications']}\n"
        f"Required Food: {disease['Required Food']}\n"
        f"Safety Precautions: {disease['Safety Precautions']}\n"
        f"Recommended Doctor: {disease['Recommended Doctor']}\n"
        f"Treatment Plan: {disease['Treatment Plan']}\n"
        f"Follow-Up Recommendations: {disease['Follow-Up Recommendations']}\n"
        f"Patient Education: {disease['Patient Education']}\n"
        f"Recovery Time: {disease['Recovery Time']}"
    )
    if language != 'en':
        translator = GoogleTranslator(source='en', target=language)
        disease_info = translator.translate(disease_info)
    print(disease_info)

# Detect language and perform sentiment analysis
def detect_language_and_sentiment(text):
    client = authenticate_azure_nlp()
    
    # Detect Language
    detected_language = client.detect_language(documents=[text])[0]
    detected_lang_code = detected_language.primary_language.iso6391_name
    print(f"Detected Language: {lang_code_to_full.get(detected_lang_code, detected_lang_code)}")
    
    # Sentiment Analysis
    sentiment_response = client.analyze_sentiment(documents=[text])[0]
    sentiment = sentiment_response.sentiment
    print(f"Sentiment: {sentiment}")
    return sentiment, detected_lang_code

# Language code to full name dictionary
lang_code_to_full = {
    'en': 'English',
    'fr': 'French',
    'es': 'Spanish',
    'de': 'German',
    'it': 'Italian',
    'pt': 'Portuguese',
    'zh': 'Chinese',
    'ja': 'Japanese',
    'ko': 'Korean',
    'ar': 'Arabic',
    'ru': 'Russian',
    'hi': 'Hindi',
    'bn': 'Bengali',
    'pa': 'Punjabi',
    'jv': 'Javanese',
    'ml': 'Malayalam',
    'tr': 'Turkish',
    'vi': 'Vietnamese',
    'th': 'Thai',
    'sv': 'Swedish',
    'da': 'Danish',
    'no': 'Norwegian',
    'fi': 'Finnish',
    'cs': 'Czech',
    'pl': 'Polish',
    'uk': 'Ukrainian',
    'el': 'Greek',
    'he': 'Hebrew',
    'te': 'Telugu',
    'ur': 'Urdu',
    'kn': 'Kannada',
    'ta': 'Tamil'
}

def extract_text_from_file(file_path):
    # Extract text from .txt files
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    
    # Extract text from .pdf files
    elif file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfFileReader(file)
            text = ""
            for page_num in range(reader.numPages):
                text += reader.getPage(page_num).extract_text()
            return text
    
    # Extract text from .docx files
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        text = ""
        for para in doc.paragraphs:
            text += para.text
        return text

    # Extract text from .pptx files
    elif file_path.endswith('.pptx'):
        presentation = Presentation(file_path)
        text = ""
        for slide in presentation.slides:
            for shape in slide.shapes:
                if hasattr(shape, 'text'):
                    text += shape.text + "\n"
        return text

    # Extract text from .xlsx files
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
        text = ""
        for column in df.columns:
            text += df[column].astype(str).str.cat(sep='\n')
        return text

    # Extract text from .csv files
    elif file_path.endswith('.csv'):
        text = ""
        with open(file_path, 'r', encoding='utf-8') as file:
            csv_reader = reader(file)
            for row in csv_reader:
                text += ' '.join(row) + "\n"
        return text

    # Extract text from .json files
    elif file_path.endswith('.json'):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return json.dumps(data, indent=2)
    
    # Extract text from .xml files
    elif file_path.endswith('.xml'):
        tree = ET.parse(file_path)
        root = tree.getroot()
        return ET.tostring(root, encoding='unicode')

    return ""


def record_live_audio(output_file_path, duration=5):
    """Record live audio and save it as a WAV file."""
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)
    
    print("Recording...")
    frames = []
    
    for _ in range(int(44100 / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)
    
    print("Recording finished.")
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    with wave.open(output_file_path, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
        wf.setframerate(44100)
        wf.writeframes(b''.join(frames))
    print(f"Recorded audio saved to {output_file_path}")
    
    return output_file_path


def transcribe_audio(file_path):
    speech_config = speechsdk.SpeechConfig(subscription=SPEECH_KEY, region=SPEECH_REGION)
    audio_config = speechsdk.AudioConfig(filename=file_path)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    
    all_text = []
    recognition_finished = False

    def recognized_callback(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            print(f"Recognized: {evt.result.text}")
            all_text.append(evt.result.text)
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized.")

    def session_stopped_callback(evt):
        nonlocal recognition_finished
        recognition_finished = True
        print("Recognition session stopped.")

    speech_recognizer.recognized.connect(recognized_callback)
    speech_recognizer.session_stopped.connect(session_stopped_callback)
    
    print("Transcribing...")
    speech_recognizer.start_continuous_recognition()

    try:
        # Wait until recognition is finished
        while not recognition_finished:
            time.sleep(1)
    except KeyboardInterrupt:
        # Stop recognition when Ctrl+C is pressed
        speech_recognizer.stop_continuous_recognition()
    
    # Ensure continuous recognition has stopped
    speech_recognizer.stop_continuous_recognition()

    return " ".join(all_text)


# Classify image using Azure Custom Vision
def classify_image(image_path):
    with open(image_path, "rb") as image_data:
        results = predictor.classify_image(project_id, publish_name, image_data.read())
    
    logging.info(f"Predictions for image: {image_path}")
    for prediction in results.predictions:
        logging.info(f"Tag: {prediction.tag_name}, Probability: {prediction.probability:.2f}")

    best_prediction = max(results.predictions, key=lambda p: p.probability)
    return best_prediction.tag_name if best_prediction.probability > 0 else None



def main():
    # Provide user with options
    choice = input("Choose an option:\n1. Enter symptoms directly\n2. Upload a file (txt, pdf, docx, pptx, xlsx, csv, json, xml)\n3. Upload an audio file\n4. Record live audio\n5. Upload an image\nEnter your choice (1, 2, 3, 4, or 5): ").strip()

    if choice == '1':
        # Option 1: User enters symptoms directly
        user_input = input("Enter your symptoms (in any language): ").strip()
        
        # Translate text to English if needed
        translated_text = translate_to_english(user_input)
        print(f"\nTranslated Text: {translated_text}")
        
        # Detect language and perform sentiment analysis
        sentiment, detected_language = detect_language_and_sentiment(translated_text)
        
        # If sentiment is positive, acknowledge and skip disease identification
        if sentiment == 'positive':
            print("The input statement is positive. No disease identification needed.")
            return
        
        # Extract key phrases (symptoms) from translated text
        key_phrases = extract_key_phrases(translated_text)
        print(f"\nExtracted Key Phrases (Symptoms): {key_phrases}")
        
        # Find diseases based on the extracted symptoms
        matched_diseases = find_disease_by_symptoms(key_phrases, combined_df)
        
        # Prioritize diseases
        if not matched_diseases.empty:
            prioritized_diseases = prioritize_combined(matched_diseases, key_phrases)
            top_disease = prioritized_diseases.iloc[0]
            
            # Ask user for their preferred output language
            preferred_language = input("Enter your preferred output language (e.g., 'fr' for French, 'es' for Spanish): ").strip().lower()
            if preferred_language not in lang_code_to_full:
                preferred_language = 'en'  # Default to English if an invalid code is provided
            
            print_disease_info(top_disease, preferred_language)
        else:
            print("\nNo matching diseases found based on the provided symptoms.")
    
    elif choice == '2':
        # Option 2: User uploads a file
        file_path = input("Enter the path to your file (txt, pdf, docx, pptx, xlsx, csv, json, xml): ").strip()
        
        if not os.path.exists(file_path):
            print("The specified file does not exist.")
            return
        
        # Extract text from the uploaded file
        file_text = extract_text_from_file(file_path)
        print(f"\nExtracted Text: {file_text}")
        
        # Translate text to English if needed
        translated_text = translate_to_english(file_text)
        print(f"\nTranslated Text: {translated_text}")
        
        # Detect language and perform sentiment analysis
        sentiment, detected_language = detect_language_and_sentiment(translated_text)
        
        # If sentiment is positive, acknowledge and skip disease identification
        if sentiment == 'positive':
            print("The input statement is positive. No disease identification needed.")
            return
        
        # Extract key phrases (symptoms) from translated text
        key_phrases = extract_key_phrases(translated_text)
        print(f"\nExtracted Key Phrases (Symptoms): {key_phrases}")
        
        # Find diseases based on the extracted symptoms
        matched_diseases = find_disease_by_symptoms(key_phrases, combined_df)
        
        # Prioritize diseases
        if not matched_diseases.empty:
            prioritized_diseases = prioritize_combined(matched_diseases, key_phrases)
            top_disease = prioritized_diseases.iloc[0]
            
            # Ask user for their preferred output language
            preferred_language = input("Enter your preferred output language (e.g., 'fr' for French, 'es' for Spanish): ").strip().lower()
            if preferred_language not in lang_code_to_full:
                preferred_language = 'en'  # Default to English if an invalid code is provided
            
            print_disease_info(top_disease, preferred_language)
        else:
            print("\nNo matching diseases found based on the provided symptoms.")
    
    elif choice == '3':
        # Option 3: User uploads an audio file
        audio_file_path = input("Enter the path to the audio file for transcription: ").strip()
        
        if not os.path.exists(audio_file_path):
            print("The specified audio file does not exist.")
            return
        
        # Transcribe audio to text
        transcription = transcribe_audio(audio_file_path)
        print(f"\nTranscription: {transcription}")
        
        # Translate text to English if needed
        translated_text = translate_to_english(transcription)
        print(f"\nTranslated Text: {translated_text}")
        
        # Detect language and perform sentiment analysis
        sentiment, detected_language = detect_language_and_sentiment(translated_text)
        
        # If sentiment is positive, acknowledge and skip disease identification
        if sentiment == 'positive':
            print("The input statement is positive. No disease identification needed.")
            return
        
        # Extract key phrases (symptoms) from translated text
        key_phrases = extract_key_phrases(translated_text)
        print(f"\nExtracted Key Phrases (Symptoms): {key_phrases}")
        
        # Find diseases based on the extracted symptoms
        matched_diseases = find_disease_by_symptoms(key_phrases, combined_df)
        
        # Prioritize diseases
        if not matched_diseases.empty:
            prioritized_diseases = prioritize_combined(matched_diseases, key_phrases)
            top_disease = prioritized_diseases.iloc[0]
            
            # Ask user for their preferred output language
            preferred_language = input("Enter your preferred output language (e.g., 'fr' for French, 'es' for Spanish): ").strip().lower()
            if preferred_language not in lang_code_to_full:
                preferred_language = 'en'  # Default to English if an invalid code is provided
            
            print_disease_info(top_disease, preferred_language)
        else:
            print("\nNo matching diseases found based on the provided symptoms.")
    
    elif choice == '4':
        output_file_path = "live_recorded_audio.mp3"
        duration = int(input("Enter the duration for live recording in seconds: ").strip())
        final_live_record_path=record_live_audio(output_file_path, duration)
        # Transcribe audio to text
        transcription = transcribe_audio(final_live_record_path)
        print(f"\nTranscription: {transcription}")
        
        # Translate text to English if needed
        translated_text = translate_to_english(transcription)
        print(f"\nTranslated Text: {translated_text}")
        
        # Detect language and perform sentiment analysis
        sentiment, detected_language = detect_language_and_sentiment(translated_text)
        
        # If sentiment is positive, acknowledge and skip disease identification
        if sentiment == 'positive':
            print("The input statement is positive. No disease identification needed.")
            return
        
        # Extract key phrases (symptoms) from translated text
        key_phrases = extract_key_phrases(translated_text)
        print(f"\nExtracted Key Phrases (Symptoms): {key_phrases}")
        
        # Find diseases based on the extracted symptoms
        matched_diseases = find_disease_by_symptoms(key_phrases, combined_df)
        
        # Prioritize diseases
        if not matched_diseases.empty:
            prioritized_diseases = prioritize_combined(matched_diseases, key_phrases)
            top_disease = prioritized_diseases.iloc[0]
            
            # Ask user for their preferred output language
            preferred_language = input("Enter your preferred output language (e.g., 'fr' for French, 'es' for Spanish): ").strip().lower()
            if preferred_language not in lang_code_to_full:
                preferred_language = 'en'  # Default to English if an invalid code is provided
            
            print_disease_info(top_disease, preferred_language)
        else:
            print("\nNo matching diseases found based on the provided symptoms.")

    elif choice == '5':
        image_path = input("Enter the path to the image file for classification: ").strip()
        if not os.path.exists(image_path):
            print("The specified image file does not exist.")
            return

        disease_tag = classify_image(image_path)
        if disease_tag:
            print(f"\nClassified Disease Tag: {disease_tag}")
            matched_diseases = combined_df[combined_df['Disease Name'].str.lower() == disease_tag.lower()]  # Use the correct column for matching


            if not matched_diseases.empty:
                prioritized_diseases = prioritize_combined(matched_diseases, [])
                top_disease = prioritized_diseases.iloc[0]
                
                preferred_language = input("Enter your preferred output language (e.g., 'fr' for French): ").strip().lower()
                if preferred_language not in lang_code_to_full:
                    preferred_language = 'en'
                print_disease_info(top_disease, preferred_language)
            else:
                print("\nNo matching diseases found for the classified tag.")
        else:
            print("No disease could be classified from the image.")

    else:
        print("Invalid choice. Please enter 1, 2, 3, 4, or 5.")

if __name__ == "__main__":
    main()


#D:\AI-Projects\FinalProject\P2\ImageClassification\BrainTumorAndChestX_RayDatasets\Training\covid19\COVID19(2).jpg
#D:\AI-Projects\FinalProject\P2\ImageClassification\BrainTumorAndChestX_RayDatasets\Training\glioma\Tr-gl_0012.jpg
#D:\AI-Projects\FinalProject\P2\ImageClassification\BrainTumorAndChestX_RayDatasets\Training\meningioma\Tr-me_0012.jpg
#D:\AI-Projects\FinalProject\P2\ImageClassification\BrainTumorAndChestX_RayDatasets\Training\notumor\Tr-no_0045.jpg
#D:\AI-Projects\FinalProject\P2\ImageClassification\BrainTumorAndChestX_RayDatasets\Training\pituitary\Tr-pi_0033.jpg    
#D:\AI-Projects\FinalProject\P2\Patientpk.wav
#D:\AI-Projects\FinalProject\P2\RecieptInJapanese.txt
#Последние несколько дней я испытываю настоящие страдания. Всё вокруг кажется серым и мрачным, и я чувствую себя одиноко и подавленно. Каждое утро начинается с тяжёлого чувства тревоги, и с каждым днём становится всё труднее справляться с этим. Надеюсь, что скоро наступят лучше времена и я смогу найти покой.

DEBUG:msrest.universal_http.requests:Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90


Choose an option:
1. Enter symptoms directly
2. Upload a file (txt, pdf, docx, pptx, xlsx, csv, json, xml)
3. Upload an audio file
4. Record live audio
5. Upload an image
Enter your choice (1, 2, 3, 4, or 5):  1
Enter your symptoms (in any language):  happy


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): translate.google.com:443
DEBUG:urllib3.connectionpool:https://translate.google.com:443 "GET /m?tl=en&sl=fi&q=happy HTTP/11" 200 None
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://p2languageservice.cognitiveservices.azure.com/language/:analyze-text?api-version=2023-04-01'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '132'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'f87f25c9-7f65-11ef-8101-914a55c7109a'
    'User-Agent': 'azsdk-python-ai-textanalytics/5.3.0 Python/3.9.11 (Windows-10-10.0.22631-SP0)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
A body is sent with the request


Detected Language: Finnish

Translated Text: happy


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): p2languageservice.cognitiveservices.azure.com:443
DEBUG:urllib3.connectionpool:https://p2languageservice.cognitiveservices.azure.com:443 "POST /language/:analyze-text?api-version=2023-04-01 HTTP/11" 200 206
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '206'
    'Content-Type': 'application/json; charset=utf-8'
    'csp-billing-usage': 'REDACTED'
    'x-envoy-upstream-service-time': '6'
    'Set-Cookie': 'REDACTED'
    'apim-request-id': 'a9514a49-2a65-4158-b1d2-b0c554f7801c'
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'
    'x-content-type-options': 'nosniff'
    'x-ms-region': 'REDACTED'
    'Date': 'Mon, 30 Sep 2024 19:55:41 GMT'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://p2languageservice.cognitiveservices.azure.com/language/:analyze-text?api-version=2023-04-01'
Request method: 'POST'


Detected Language: English


DEBUG:urllib3.connectionpool:https://p2languageservice.cognitiveservices.azure.com:443 "POST /language/:analyze-text?api-version=2023-04-01 HTTP/11" 200 351
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '351'
    'Content-Type': 'application/json; charset=utf-8'
    'csp-billing-usage': 'REDACTED'
    'x-envoy-upstream-service-time': '12'
    'Set-Cookie': 'REDACTED'
    'apim-request-id': '4772be60-c423-4be6-8d39-3e1fd3137e7a'
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'
    'x-content-type-options': 'nosniff'
    'x-ms-region': 'REDACTED'
    'Date': 'Mon, 30 Sep 2024 19:55:41 GMT'


Sentiment: positive
The input statement is positive. No disease identification needed.
